In [ ]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import geometry as geom
import plotly.plotly as py
import sys
 

#added this because it's indicated as useful but not imported
import urllib
from zipfile import ZipFile
from io import BytesIO
import os
import seaborn as sns
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup as bs
import re

from matplotlib import animation, rc
from IPython.display import display, HTML

In [ ]:
with open("keys.json") as json_file:    
    keys = json.load(json_file)
census_key = keys["census"]["api_key"]

In [ ]:
census_vars = json.loads(urllib.request.urlopen("http://api.census.gov/data/2014/acs5/variables.json").read().decode('utf-8'))["variables"]
income_brackets = dict([(key, value["label"]) for key, value in census_vars.items() if "B19001_" in key and key[-1] == "E"])

In [ ]:
from us import states
from census import Census

def getincome20xx(income_brackets, years, census_key, zips):
    c = Census(census_key)
    df = pd.DataFrame()
    for year in years:
        tempget = c.acs5.zipcode((tuple([key[0] for key in income_brackets.items()])), Census.ALL, year=year)
        tempDF = pd.DataFrame(tempget)
        tempDF.columns = [income_brackets[tempDF.columns[i]] if i < len(income_brackets) else tempDF.columns[i] for i in range(len(tempDF.columns))]
        tempDF = tempDF[tempDF[tempDF.columns[-1]].isin(zips)]
        tempDF["year"] = year
        df = df.append(tempDF)
    return df

I forgot I wrote this. dope. :D

In [ ]:
zipgeojson = "https://github.com/YuHuaCheng/Map/raw/master/data/nyc-zip-code-tabulation-areas-polygons.geojson"

# save the NYC map data under "data" folder
zipsfile = os.path.join("data",zipgeojson.split("/")[-1])

# read the saved NYC map data file
try:
    NY = gpd.read_file(zipsfile) # NYC map data with Zipcode
except OSError as e:
    urllib.request.urlretrieve(zipgeojson, zipsfile)
    
    #zips geojson had a var assignment that caused problems
    fin      = open(zipsfile)
    
    lines    = [line for line in fin]
    lines[0] = lines[0].replace("var nycdata = ", "")
    
    fin.close()
    
    fout = open(zipsfile, "w")
    for line in lines:
        fout.write(line)
    fout.flush()
    fout.close()
    
    NY = gpd.read_file(zipsfile)
    
NY.shape # NYC is composed of 262 subarea on the map data

In [ ]:
QN = NY[NY.borough == "Queens"]
zips_QN = QN.postalCode.unique()

In [ ]:
theDF = getincome20xx(income_brackets,range(2011,2015), census_key, zips_QN)

In [ ]:
newcols = ["total", "0", "10", "15", "20", "25", "30", "35", "40", "45", "50", "60", "75", "100", "125", "150", "200", "ZIP", "year"]
theDF.columns = newcols
theDF[newcols] = theDF[newcols].apply(pd.to_numeric)
theDF.head()

In [ ]:
theDF.to_csv(os.path.join('data', "QN_incomeDF.csv"))

In [ ]:
QN_files = ["https://www.huduser.gov/portal/datasets/fmr/fmr2010f/FY2010_4050_Final_PostRDDs.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2011f/fy2011_equiv_zc_rents_acs.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2012f/small_area_fmrs_fy2012.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2013p/small_area_fmrs_fy2013.xls",
         "https://www.huduser.gov/portal/datasets/fmr/fmr2014f/small_area_fmrs_fy2014.xls"]

In [ ]:
from JP_Cache import get_cached_or_dl
from IPython.display import display

In [ ]:
rent_per_year = get_cached_or_dl(QN_files)

In [ ]:
for i in range(len(rent_per_year)):
    for j in rent_per_year[i].columns.values:
        if j == 'zip' or j == 'ZIP':
            rent_per_year[i].rename(columns={j: 'ZIP' }, inplace=True)
        if j == 'countyname' or j == 'County Name' or j == 'County_Name':
            rent_per_year[i].rename(columns={j: 'CountyName' }, inplace=True)
        if j == 'State Name' or j == 'state_alpha':
            rent_per_year[i].rename(columns={j: 'StateName' }, inplace=True)
        if j == 'area_rent_br0' or j == 'fmr0':
            rent_per_year[i].rename(columns={j: 'rent0' }, inplace=True)
        if j == 'area_rent_br1' or j == 'fmr1':
            rent_per_year[i].rename(columns={j: 'rent1' }, inplace=True)
        if j == 'area_rent_br2' or j == 'fmr2':
            rent_per_year[i].rename(columns={j: 'rent2' }, inplace=True)
        if j == 'area_rent_br3' or j == 'fmr3':
            rent_per_year[i].rename(columns={j: 'rent3' }, inplace=True)
        if j == 'area_rent_br4' or j == 'fmr4':
            rent_per_year[i].rename(columns={j: 'rent4' }, inplace=True)
        if j == 'state':
            rent_per_year[i].rename(columns={j: 'State' }, inplace=True)

In [ ]:
#df_14 = 
#df_14 = df_14[['StateName','CountyName','rent0','rent1','rent2','rent3','rent4','ZIP']]
#df_14.head()

In [ ]:
rent_per_year[1]["year"] = 2011
rent_per_year[2]["year"] = 2012
rent_per_year[3]["year"] = 2013
rent_per_year[4]["year"] = 2014
rent_per_year = rent_per_year[1:] # drop 2010

In [ ]:
rent_per_year_QN = []
for rentdf in rent_per_year:
    #display(rentdf.head())
    rent_per_year_QN.append(rentdf[(rentdf.CountyName == 'Queens County') & (rentdf.State == 36)])

In [ ]:
rent_per_year_QN[0]["StateName"] = "New York"

In [ ]:
for rentdf in rent_per_year_QN:
    display(rentdf.head())

In [ ]:
rentQN = pd.DataFrame()
for rentdf in rent_per_year_QN:
    tempdf = rentdf[['State','CountyName','rent0','rent1','rent2','rent3','rent4','ZIP', 'year']]
    print((len(rentdf), len(tempdf)))
    rentQN = rentQN.append(tempdf, ignore_index=True)
print(len(rentQN))
rentQN.head()

In [ ]:
rentQN[['rent0','rent1','rent2','rent3','rent4',"ZIP", "year"]] = rentQN[['rent0','rent1','rent2','rent3','rent4',"ZIP", "year"]].apply(pd.to_numeric)
rentQN.head()

In [ ]:
rentQN.to_csv(os.path.join('data', "QN_rentDF.csv"))

In [ ]:
merged = pd.merge(theDF,rentQN,how = "inner", on =['year', 'ZIP'])
merged.head()

In [ ]:
merged.to_csv("data/QN_merged.csv")

In [ ]:
theDF.head()

Tutorial: http://seaborn.pydata.org/tutorial.html

In [ ]:

fd = merged.groupby("year").size().plot(kind="bar", figsize=(6, 6))

fd.set_yticklabels([])  # turn off y tick labels

# resize x label
xlabel = fd.xaxis.get_label()
xlabel.set_fontsize(24)

# resize x tick labels
lb = fd.xaxis.get_ticklabels()
[label.set_fontsize(24) for label in lb];

In [ ]:
g = sns.FacetGrid(merged, col="year", size=20, aspect=1)
g.map(sns.barplot, "ZIP", "total")
g.set_xticklabels(rotation=90)

In [ ]:
sns.factorplot("ZIP", "year", hue="total", col="45", data=merged)